In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data4 = data[data['PdlCyclist'] == 1]
filtered_data4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12354 entries, 32 to 54035
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       12354 non-null  int64  
 1   BedDys           12354 non-null  int64  
 2   BeddystoCC       12354 non-null  float64
 3   Severity         12354 non-null  int64  
 4   DiedCases        12354 non-null  int64  
 5   DiedBdDys        12354 non-null  int64  
 6   Hsptlsns         12354 non-null  int64  
 7   April            12354 non-null  int64  
 8   August           12354 non-null  int64  
 9   December         12354 non-null  int64  
 10  February         12354 non-null  int64  
 11  January          12354 non-null  int64  
 12  July             12354 non-null  int64  
 13  June             12354 non-null  int64  
 14  March            12354 non-null  int64  
 15  May              12354 non-null  int64  
 16  November         12354 non-null  int64  
 17  October         

In [4]:
num_bins = 2

bed_days_min = filtered_data4['Severity'].min()
bed_days_max = filtered_data4['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_2200/3159902379.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data4['Bed_days_category'] = filtered_data4['Severity'].apply(bin_function)


In [5]:
from xgboost import XGBClassifier

X = filtered_data4[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data4['Bed_days_category']

In [6]:
y.value_counts()

Bed_days_category
0    12262
1       92
Name: count, dtype: int64

LOGESTIC REGRESSION

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks
from imblearn.over_sampling import SMOTE
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Logistic Regression model without undersampling
    logreg_model = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = logreg_model.predict(X_test)
    y_prob_before = logreg_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model with NearMiss
    logreg_model_nearmiss = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = logreg_model_nearmiss.predict(X_test)
    y_prob_nearmiss = logreg_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model with ClusterCentroids
    logreg_model_clustercentroids = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = logreg_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = logreg_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model with TomekLinks
    logreg_model_tomeklinks = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = logreg_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = logreg_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)


Confusion Matrix for Fold 1 Before Undersampling:
 [[1224    3]
 [   2    7]]
Confusion Matrix for Fold 1 After NearMiss:
 [[975 252]
 [  0   9]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[1038  189]
 [   0    9]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[1223    4]
 [   2    7]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 2 After NearMiss:
 [[993 234]
 [  0   9]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[1060  167]
 [   0    9]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[1223    3]
 [   5    5]]
Confusion Matrix for Fold 3 After NearMiss:
 [[967 259]
 [  1   9]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[1030  196]
 [   0   10]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[1223    3]
 [   3    7]]
Confusion Matrix for Fold 4 Before Undersampling:
 [[1224    2]
 [   3    7]]
Confusion Matrix for F

XGBOOST

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model without undersampling
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with NearMiss
    xgb_model_nearmiss = XGBClassifier(random_state=42)
    xgb_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = xgb_model_nearmiss.predict(X_test)
    y_prob_nearmiss = xgb_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with ClusterCentroids
    xgb_model_clustercentroids = XGBClassifier(random_state=42)
    xgb_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = xgb_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = xgb_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with TomekLinks
    xgb_model_tomeklinks = XGBClassifier(random_state=42)
    xgb_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = xgb_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = xgb_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)


Confusion Matrix for Fold 1 Before Undersampling:
 [[1223    4]
 [   1    8]]
Confusion Matrix for Fold 1 After NearMiss:
 [[1001  226]
 [   0    9]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[924 303]
 [  0   9]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[1223    4]
 [   2    7]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 2 After NearMiss:
 [[1014  213]
 [   0    9]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[1091  136]
 [   0    9]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[1225    2]
 [   0    9]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[1224    2]
 [   3    7]]
Confusion Matrix for Fold 3 After NearMiss:
 [[990 236]
 [  3   7]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[1081  145]
 [   0   10]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[1223    3]
 [   4    6]]
Confusion Matrix for Fold 4 Before Undersampling:
 [[1224    2]
 [   1    9]]
Confusion Matrix f

svm

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the SVM model without undersampling
    svm_model = SVC(probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = svm_model.predict(X_test)
    y_prob_before = svm_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the SVM model with NearMiss
    svm_model_nearmiss = SVC(probability=True, random_state=42)
    svm_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = svm_model_nearmiss.predict(X_test)
    y_prob_nearmiss = svm_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the SVM model with ClusterCentroids
    svm_model_clustercentroids = SVC(probability=True, random_state=42)
    svm_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = svm_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = svm_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the SVM model with TomekLinks
    svm_model_tomeklinks = SVC(probability=True, random_state=42)
    svm_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = svm_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = svm_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)


Confusion Matrix for Fold 1 Before Undersampling:
 [[1222    5]
 [   0    9]]
Confusion Matrix for Fold 1 After NearMiss:
 [[643 584]
 [  0   9]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[874 353]
 [  0   9]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[1222    5]
 [   0    9]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[1224    3]
 [   0    9]]
Confusion Matrix for Fold 2 After NearMiss:
 [[653 574]
 [  0   9]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[1002  225]
 [   0    9]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[1224    3]
 [   0    9]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[1221    5]
 [   2    8]]
Confusion Matrix for Fold 3 After NearMiss:
 [[505 721]
 [  1   9]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[903 323]
 [  0  10]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[1221    5]
 [   2    8]]
Confusion Matrix for Fold 4 Before Undersampling:
 [[1224    2]
 [   0   10]]
Confusion Matrix for Fold 4 Af

Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Random Forest model without undersampling
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model with NearMiss
    rf_model_nearmiss = RandomForestClassifier(random_state=42)
    rf_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = rf_model_nearmiss.predict(X_test)
    y_prob_nearmiss = rf_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model with ClusterCentroids
    rf_model_clustercentroids = RandomForestClassifier(random_state=42)
    rf_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = rf_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = rf_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model with TomekLinks
    rf_model_tomeklinks = RandomForestClassifier(random_state=42)
    rf_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = rf_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = rf_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)


Confusion Matrix for Fold 1 Before Undersampling:
 [[1223    4]
 [   3    6]]
Confusion Matrix for Fold 1 After NearMiss:
 [[997 230]
 [  1   8]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[1067  160]
 [   0    9]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[1223    4]
 [   4    5]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[1226    1]
 [   3    6]]
Confusion Matrix for Fold 2 After NearMiss:
 [[1035  192]
 [   3    6]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[1146   81]
 [   0    9]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[1226    1]
 [   3    6]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[1224    2]
 [   5    5]]
Confusion Matrix for Fold 3 After NearMiss:
 [[996 230]
 [  4   6]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[1110  116]
 [   0   10]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[1223    3]
 [   3    7]]
Confusion Matrix for Fold 4 Before Undersampling:
 [[1225    1]
 [   4    6]]
Confusion Matrix f

decesion tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Decision Tree model without undersampling
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = dt_model.predict(X_test)
    y_prob_before = dt_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model with NearMiss
    dt_model_nearmiss = DecisionTreeClassifier(random_state=42)
    dt_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = dt_model_nearmiss.predict(X_test)
    y_prob_nearmiss = dt_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model with ClusterCentroids
    dt_model_clustercentroids = DecisionTreeClassifier(random_state=42)
    dt_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = dt_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = dt_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model with TomekLinks
    dt_model_tomeklinks = DecisionTreeClassifier(random_state=42)
    dt_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = dt_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = dt_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)


Confusion Matrix for Fold 1 Before Undersampling:
 [[1223    4]
 [   6    3]]
Confusion Matrix for Fold 1 After NearMiss:
 [[838 389]
 [  1   8]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[889 338]
 [  0   9]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[1221    6]
 [   6    3]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[1224    3]
 [   4    5]]
Confusion Matrix for Fold 2 After NearMiss:
 [[842 385]
 [  3   6]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[1029  198]
 [   0    9]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[1223    4]
 [   2    7]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[1224    2]
 [   5    5]]
Confusion Matrix for Fold 3 After NearMiss:
 [[985 241]
 [  5   5]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[876 350]
 [  1   9]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[1224    2]
 [   5    5]]
Confusion Matrix for Fold 4 Before Undersampling:
 [[1223    3]
 [   3    7]]
Confusion Matrix for Fold 4 Af